In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import HDF5Matrix 
import h5py
%matplotlib inline

Using TensorFlow backend.


In [ ]:
#f= h5py.File('Train/food_c101_n1000_r384x384x3.h5','r')

In [ ]:
xtrain=HDF5Matrix('Train/food_c101_n10099_r32x32x1.h5','images')
ytrain=HDF5Matrix('Train/food_c101_n10099_r32x32x1.h5','category')

In [ ]:
print ('xtrain shape is: ',xtrain.shape,', ytrain shape is: ',ytrain.shape)

In [ ]:
xtrain=np.asarray(xtrain,dtype='float32')


In [ ]:
ytrain=np.asarray(ytrain,dtype='float32')

In [ ]:
xtest=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','images')
ytest=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','category')

In [ ]:
print ('xtest shape is: ',xtest.shape,', ytest shape is: ',ytest.shape)

In [ ]:
xtest=np.asarray(xtest,dtype='float32')
ytest=np.asarray(ytest,dtype='float32')

In [ ]:
labels =f.get('category')

In [ ]:
labels[:][:].shape

In [ ]:
labels=np.array(labels)

In [ ]:
header=np.array(f)

In [ ]:
header

In [ ]:
category_names =f.get('category_names')
category_names=np.array(category_names)

In [ ]:
category_names.shape

scale the data  

In [ ]:
xtrain_scaled=xtrain/255
xtest_scaled=xtest/255

# encode the labels

In [ ]:
#from keras.utils import to_categorical

In [ ]:
#labels_encoded=to_categorical(labels)

In [ ]:
#labels_encoded=labels_encoded.astype('float32')

In [ ]:
#labels_encoded.T.shape

# Network params

In [ ]:
n_channel=1
n_width=32
n_heght=32
n_output=101

In [ ]:
def create_variable(channel,width,height,output):
    X=tf.placeholder(tf.float32,shape=[None,height,width,channel],name='Input')
    Y=tf.placeholder(tf.float32,shape=[None,output],name='output')
    return X,Y

In [ ]:
def initialize_variable():
    W1=tf.get_variable("W1",[4,4,1,8],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    W2=tf.get_variable("W2",[4,4,8,16],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    return W1,W2
    

In [ ]:
#Create mini-batch

def mini_batch(xdata,ydata,batch_size):
    x_size=xdata.shape[0]
    y_size=ydata.shape[0]
    for i in range(0,x_size,batch_size):
        yield xdata[i:i+batch_size],ydata[i:i+batch_size]
    

CNN Layers

In [ ]:
def create_layer(W1,W2,X,Y):
    #W1,W2=initialize_variable()
    #X,Y=create_variable(channel,width,height,output)
    conv1=tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
    max_pool1=tf.nn.max_pool(tf.nn.relu(conv1),ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
    
    conv2=tf.nn.conv2d(max_pool1,W2,strides=[1,2,2,1],padding='SAME')
    max_pool2=tf.nn.max_pool(tf.nn.relu(conv2),ksize=[1,4,4,1],strides=[1,2,2,1],padding='SAME')
    
    max_pool2=tf.contrib.layers.flatten(max_pool2)
    out=tf.contrib.layers.fully_connected(max_pool2,n_output,activation_fn=None)
    
    return out

In [ ]:
def compute_cost(out,Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out,labels=Y))
    return cost

In [ ]:
def model():
    learning_rate=0.05
    tf.reset_default_graph()
    #tf.get_default_graph()
    
    xtrain_initialiazer=tf.placeholder(dtype=xtrain.dtype,shape=xtrain.shape)
    ytrain_initialiazer=tf.placeholder(dtype=ytrain.dtype,shape=ytrain.shape)
    
    xtrain_input=tf.Variable(xtrain_initialiazer,trainable=False,collections=[])
    ytrain_input=tf.Variable(ytrain_initialiazer,trainable=False,collections=[])
    
    images,labels=tf.train.slice_input_producer([xtrain_input,ytrain_input],num_epochs=2)
    labels=tf.cast(labels,tf.float32)
    
    images,labels=tf.train.batch([images,labels],batch_size=128)
    
    
    #x,y=create_variable(n_channel,n_width,n_heght,n_output)
    W1,W2=initialize_variable()
    out = create_layer(W1,W2,images,labels)
    loss=compute_cost(out,labels)
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    #accuracy
    
    
    init_op=tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    
    with tf.Session() as sess:
        sess.run(init_op)
        sess.run(xtrain_input.initializer,feed_dict={xtrain_initialiazer:xtrain_scaled})
        sess.run(ytrain_input.initializer,feed_dict={ytrain_initialiazer:ytrain})
        
            
        #start the enque thread
        coord=tf.train.Coordinator()
        threads=tf.train.start_queue_runners(sess,coord=coord)
        while not coord.should_stop() :
            
            opt,cost=sess.run([optimizer,loss])   
        #coord.request_stop()
        correct_pred=tf.equal(tf.argmax(out,axis=1),tf.argmax(labels,axis=1))
        accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))
        print('accuracy',accuracy)
        print("Test accuracy",accuracy.eval())
    return opt,cost
    
    

In [ ]:

model()

In [ ]:
type(xtrain_scaled)

In [ ]:
type(ytrain)

In [ ]:
ytrain.shape